# S&P 500 index prediction

`Dataset to prepare: (Data name(source))` 
* historical S&P index (Yahoo Finance)
* oil & gas price (Yahoo Finance)
* unemployment rate (US Bureau of Labor)
* prime rate (banks)
* Other stock exchange index(Hang seng, Zurich)
* trading volume (Yahoo Finance)
* CPI (US Bureau of Labor)
* Bond yield(treasury bond/bill) (Yahoo Finance)
* Consumer_Sentiment_Index(Nasdaq)
* gold price
* Hangseng_index
* S&P monthly Price/Earning Ratio(Nasdaq)
* S&P monthly Dividend rate(Nasdaq)
* S&P monthly earning yield ratio(Nasdaq)
* +a: future price (crude oil etc)     

#### These dataset are separate csv files, so first of all we need to merge them into 1 dataframe.

### 1. Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

### Monthly data

#### Almost same process as daily data, but there are some difference described below.

### 1&2. Importing libraries and data

In [63]:
# Oldest date is 1928-01-06 (Volume records since 1951-Dec-24)
SP_monthly = pd.read_csv('data/Raw_data/SP_monthly.csv')
SP_monthly['Date'] = pd.to_datetime(SP_monthly['Date'])
SP_monthly = SP_monthly.drop(['High','Low','Adj Close**'], axis=1)
SP_monthly = SP_monthly.rename(columns = {'Open':'S&P_Open','Close*':'S&P_Close','Volume':'S&P_Volume','Date_adj':'Date'})
SP_monthly = SP_monthly.set_index('Date')
SP_monthly.replace(to_replace=',', value='', inplace=True, regex = True)
SP_monthly.replace(to_replace='-', value=np.nan, inplace=True)
SP_monthly.head(2)

,S&P_Open,S&P_Close,S&P_Volume
Date,,,
2022-09-01,3936.73,3908.19,7889490000
2022-08-01,4112.38,3955.00,78088520000


Since September 2022 is not a full month data, I will get rid of it.

In [68]:
SP_monthly.drop(['2022-09-01'], inplace=True)
SP_monthly.head(2)

,S&P_Open,S&P_Close,S&P_Volume
Date,,,
2022-08-01,4112.38,3955.00,7.808852e+10
2022-07-01,3781.00,4130.29,6.802114e+10


In [69]:
SP_monthly = SP_monthly.astype(float)
SP_monthly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 452 entries, 2022-08-01 to 1985-01-01
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   S&P_Open    452 non-null    float64
 1   S&P_Close   452 non-null    float64
 2   S&P_Volume  452 non-null    float64
dtypes: float64(3)
memory usage: 14.1 KB


### 3. Data Combining

In [83]:
Russell = pd.read_csv('data/Raw_data/Russell2000_monthly.csv')
Russell = Russell.drop(['High','Low','Adj Close'], axis=1)
Russell['Date']= pd.to_datetime(Russell['Date'])
Russell = Russell.rename(columns = {'Open':'R_Open','Close':'R_Close','Volume':'R_Volume'})
Russell.head(2)

,Date,R_Open,R_Close,R_Volume
0,1987-10-01,170.820007,118.260002,6094500000
1,1987-11-01,118.269997,111.699997,3589520000


In [84]:
gold_future = pd.read_csv('data/Raw_data/gold_future_monthly.csv')
gold_future = gold_future.drop(['High','Low','Adj Close**'], axis=1)
gold_future['Date']= pd.to_datetime(gold_future['Date'])
gold_future = gold_future.rename(columns = {'Open':'GF_Open','Close*':'GF_Close','Volume':'GF_Volume'})
gold_future = gold_future.set_index('Date')
gold_future.replace(to_replace=',', value='', inplace=True, regex = True)
gold_future.replace(to_replace='-', value=np.nan, inplace=True)
gold_future = gold_future.astype(float)
gold_future.head(2)

,GF_Open,GF_Close,GF_Volume
Date,,,
2022-09-01,1707.9,1700.4,1047.0
2022-08-01,1763.9,1712.8,11161.0
2022-07-01,1795.5,1762.9,194989.0
2022-06-01,1831.0,1804.1,7851.0
2022-04-01,1935.0,1909.3,7725.0


In [85]:
nasdaq = pd.read_csv('data/Raw_data/Nasdaq_monthly.csv')
nasdaq = nasdaq.drop(['High','Low','Adj Close'], axis=1)
nasdaq['Date']= pd.to_datetime(nasdaq['Date'])
nasdaq = nasdaq.rename(columns = {'Open':'Nas_Open','Close':'Nas_Close','Volume':'Nas_Volume'})
nasdaq.head(2)

,Date,Nas_Open,Nas_Close,Nas_Volume
0,1985-01-01,247.100006,278.700012,1782060000
1,1985-02-01,277.899994,284.200012,1742850000


In [86]:
nikkei = pd.read_csv('data/Raw_data/Nikkei_index_monthly.csv')
nikkei = nikkei.drop(['High','Low','Adj Close'], axis=1)
nikkei['Date']= pd.to_datetime(nikkei['Date'])
nikkei = nikkei.rename(columns = {'Open':'Nk_Open','Close':'Nk_Close','Volume':'Nk_Volume'})
nikkei.head(2)

,Date,Nk_Open,Nk_Close,Nk_Volume
0,1985-01-01,11543.000000,11992.309570,0
1,1985-02-01,11946.959961,12321.919922,0


In [87]:
fx = pd.read_csv('data/Raw_data/USD_CAD Historical Data_monthly.csv')
fx = fx.drop(['Open','High','Low','Vol.'], axis=1)
fx['Date']= pd.to_datetime(fx['Date'])
fx = fx.rename(columns = {'Price':'fx_Price','Change %':'fx_change%'})
fx.replace(to_replace='%', value='', inplace=True, regex = True)
fx['fx_change%'] = fx['fx_change%'].astype(float)
fx.head(2)

,Date,fx_Price,fx_change%
0,2022-09-01,1.3140,0.10
1,2022-08-01,1.3127,2.61


In [88]:
volatility = pd.read_csv('data/Raw_data/CBOE_volatility_index_monthly.csv')
volatility = volatility.drop(['High','Low','Adj Close','Volume'], axis=1)
volatility['Date']= pd.to_datetime(volatility['Date'])
volatility = volatility.rename(columns = {'Open':'vo_Open','Close':'vo_Close'})
volatility.head(2)

,Date,vo_Open,vo_Close
0,1990-02-01,24.870001,21.99
1,1990-03-01,21.900000,19.73


In [89]:
t3m_monthly = pd.read_csv('data/Raw_data/treasury_3m_monthly.csv')
t3m_monthly['Date']= pd.to_datetime(t3m_monthly['Date'])
t3m_monthly = t3m_monthly[['Date','Close']]
t3m_monthly = t3m_monthly.rename(columns={'Close':'t_3m_Close'})
t3m_monthly.tail(2)

,Date,t_3m_Close
451,2022-08-01,2.860
452,2022-09-01,2.845


In [90]:
t10y_monthly = pd.read_csv('data/Raw_data/treasury_10y_monthly.csv')
t10y_monthly['Date']= pd.to_datetime(t10y_monthly['Date'])
t10y_monthly = t10y_monthly[['Date','Close']]
t10y_monthly = t10y_monthly.rename(columns={'Close':'t_10y_Close'})
t10y_monthly.tail(2)

,Date,t_10y_Close
451,2022-08-01,3.133
452,2022-09-01,3.340


In [92]:
oil = oil.set_index('Date')
oil_monthly = oil.resample('MS').mean()

PutCallRatio = PutCallRatio.set_index('Date')
PCR_monthly = PutCallRatio.resample('MS').mean()

In [94]:
# Merging all data into one file
Filename = [treasury_10y, cpi, oil_monthly, unemployment, SP_PER, SP_EYR, PCR_monthly, Russell, 
            gold_future, nasdaq, nikkei, fx, volatility]

SP_month = SP_monthly.merge(treasury_3m, on='Date', how='left')

for file in Filename:
    SP_month = SP_month.merge(file, on='Date', how='left')

In [95]:
SP_month.head()

,Date,S&P_Open,S&P_Close,S&P_Volume,Close_t-1,return(%),return_class,Close_t+1,t+1_return(%),t+1_return_class,...,Nas_Open,Nas_Close,Nas_Volume,Nk_Open,Nk_Close,Nk_Volume,fx_Price,fx_change%,vo_Open,vo_Close
0,2022-08-01,4112.38,3955.00,7.808852e+10,4130.29,-4.244012,-1,NaN,NaN,NaN,...,12317.959961,11816.200195,109521110000,27813.820313,28091.529297,1397800000,1.3127,2.61,22.410000,25.870001
1,2022-07-01,3781.00,4130.29,6.802114e+10,3785.38,9.111635,1,3955.00,-4.244012,-1.0,...,11006.830078,12390.690430,93619410000,26460.710938,27801.640625,1339300000,1.2793,-0.61,29.530001,21.330000
2,2022-06-01,4149.78,3785.38,8.679906e+10,4132.15,-8.391999,-1,4130.29,9.111635,1.0,...,12176.889648,11028.740234,113344650000,27295.630859,26393.039063,1637700000,1.2872,1.80,26.049999,28.709999
3,2022-05-01,4130.61,4132.15,8.912813e+10,4131.93,0.005324,0,3785.38,-8.391999,-1.0,...,12331.690430,12081.389648,110947710000,26851.099609,27279.800781,1582900000,1.2644,-1.65,33.349998,26.190001
4,2022-04-01,4540.32,4131.93,7.478485e+10,4530.41,-8.795672,-1,4132.15,0.005324,0.0,...,14269.530273,12334.639648,95994300000,27624.109375,26847.900391,1345900000,1.2856,2.86,20.620001,33.400002


In [96]:
SP_month.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452 entries, 0 to 451
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              452 non-null    datetime64[ns]
 1   S&P_Open          452 non-null    float64       
 2   S&P_Close         452 non-null    float64       
 3   S&P_Volume        452 non-null    float64       
 4   Close_t-1         451 non-null    float64       
 5   return(%)         451 non-null    float64       
 6   return_class      452 non-null    int64         
 7   Close_t+1         451 non-null    float64       
 8   t+1_return(%)     451 non-null    float64       
 9   t+1_return_class  451 non-null    float64       
 10  t_3m_Close        289 non-null    float64       
 11  t_10y_Close       289 non-null    object        
 12  cpi_Value         451 non-null    float64       
 13  cpi_MoM%          450 non-null    float64       
 14  oil_Price         440 non-

In [97]:
SP_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15272 entries, 1 to 15272
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              15272 non-null  datetime64[ns]
 1   S&P_Open          15272 non-null  float64       
 2   S&P_Close         15272 non-null  float64       
 3   S&P_Volume        15272 non-null  float64       
 4   PRIME             15272 non-null  float64       
 5   t_3m_Close        15272 non-null  float64       
 6   t_10y_Close       15272 non-null  object        
 7   cpi_Value         15272 non-null  float64       
 8   cpi_MoM%          15272 non-null  float64       
 9   unemp_rate        15272 non-null  float64       
 10  PE_Ratio          15272 non-null  float64       
 11  Div_Ratio         15272 non-null  float64       
 12  EY_Ratio          15272 non-null  float64       
 13  CS_Index          15272 non-null  float64       
 14  Close_t-1         1527

### 4. Data Cleaning

In [98]:
def year_month(date_data):
    year_month = []
    for date in date_data:
        date_year = date.year
        date_month = date.month
        year_month.append(str(date_year)+'-'+str(date_month))
    return year_month

In [100]:
SP_month['year_month'] = year_month(SP_month['Date'])

In [101]:
SP_month.head()

,Date,S&P_Open,S&P_Close,S&P_Volume,Close_t-1,return(%),return_class,Close_t+1,t+1_return(%),t+1_return_class,...,Nas_Close,Nas_Volume,Nk_Open,Nk_Close,Nk_Volume,fx_Price,fx_change%,vo_Open,vo_Close,year_month
0,2022-08-01,4112.38,3955.00,7.808852e+10,4130.29,-4.244012,-1,NaN,NaN,NaN,...,11816.200195,109521110000,27813.820313,28091.529297,1397800000,1.3127,2.61,22.410000,25.870001,2022-8
1,2022-07-01,3781.00,4130.29,6.802114e+10,3785.38,9.111635,1,3955.00,-4.244012,-1.0,...,12390.690430,93619410000,26460.710938,27801.640625,1339300000,1.2793,-0.61,29.530001,21.330000,2022-7
2,2022-06-01,4149.78,3785.38,8.679906e+10,4132.15,-8.391999,-1,4130.29,9.111635,1.0,...,11028.740234,113344650000,27295.630859,26393.039063,1637700000,1.2872,1.80,26.049999,28.709999,2022-6
3,2022-05-01,4130.61,4132.15,8.912813e+10,4131.93,0.005324,0,3785.38,-8.391999,-1.0,...,12081.389648,110947710000,26851.099609,27279.800781,1582900000,1.2644,-1.65,33.349998,26.190001,2022-5
4,2022-04-01,4540.32,4131.93,7.478485e+10,4530.41,-8.795672,-1,4132.15,0.005324,0.0,...,12334.639648,95994300000,27624.109375,26847.900391,1345900000,1.2856,2.86,20.620001,33.400002,2022-4


In [102]:
CSI['year_month'] = year_month(CSI['Date'])
SP_DIV['year_month'] = year_month(SP_DIV['Date'])

Filename = [SP_DIV, CSI]
for file in Filename:
    SP_month = SP_month.merge(file, on='year_month', how='left')

In [103]:
SP_all_monthly = SP_month.drop(['Date_x','Date_y','Date'], axis=1)
SP_all_monthly = SP_all_monthly.set_index('year_month')
SP_all_monthly.head(2)

,S&P_Open,S&P_Close,S&P_Volume,Close_t-1,return(%),return_class,Close_t+1,t+1_return(%),t+1_return_class,t_3m_Close,...,Nas_Volume,Nk_Open,Nk_Close,Nk_Volume,fx_Price,fx_change%,vo_Open,vo_Close,Div_Ratio,CS_Index
year_month,,,,,,,,,,,,,,,,,,,,,
2022-8,4112.38,3955.00,7.808852e+10,4130.29,-4.244012,-1,NaN,NaN,NaN,2.265,...,109521110000,27813.820313,28091.529297,1397800000,1.3127,2.61,22.410000,25.870001,NaN,NaN
2022-7,3781.00,4130.29,6.802114e+10,3785.38,9.111635,1,3955.0,-4.244012,-1.0,1.603,...,93619410000,26460.710938,27801.640625,1339300000,1.2793,-0.61,29.530001,21.330000,NaN,51.5


In [ ]:
#missing data
total = SP_all_monthly.isnull().sum().sort_values(ascending=False)
percent = (SP_all_monthly.isnull().sum()/SP_all_monthly.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

In [104]:
cols_tobfill = ['t_3m_Close','t_10y_Close','unemp_rate','cpi_Value',
               'cpi_MoM%','oil_Price','PE_Ratio','Div_Ratio','EY_Ratio','CS_Index','Put/Call_Ratio']

for col in cols_tobfill:
    SP_all_monthly[col].fillna(method='bfill', inplace=True)         

In [105]:
cols_toffill = ['t_3m_Close','t_10y_Close','cpi_Value','oil_Price','PE_Ratio',
                'Div_Ratio','EY_Ratio']

for col in cols_toffill:
    SP_all_monthly[col].fillna(method='ffill', inplace=True)         

In [111]:
#missing data
total = SP_all_monthly.isnull().sum().sort_values(ascending=False)
percent = (SP_all_monthly.isnull().sum()/SP_all_monthly.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
Put/Call_Ratio,306,0.676991
GF_Close,227,0.502212
GF_Open,227,0.502212
GF_Volume,227,0.502212
vo_Open,61,0.134956
vo_Close,61,0.134956
R_Volume,33,0.073009
R_Close,33,0.073009
R_Open,33,0.073009
t+1_return_class,1,0.002212


In [106]:
cols_tofill_mean = ['cpi_MoM%','unemp_rate','CS_Index','Put/Call_Ratio']
for col in cols_tofill_mean:
    SP_all_monthly[col].fillna(SP_all[col].mean(), inplace=True)

KeyError: 'Put/Call_Ratio'

In [107]:
SP_all_monthly = SP_all_monthly.drop(['year','month'], axis=1)
SP_all_monthly

KeyError: "['year', 'month'] not found in axis"

Let's set up a target variable. 
#### Target variable creation 

The goal of S&P index prediction is to predict future('t+1') index based on the current information('t'). Let's create our target variable 'Close_t+1' and 'return rate_t+1'. 

In [ ]:
SP_all_monthly['Close_t-1'] = SP_all__monthly['S&P_Close'].shift(-1)
SP_all_monthly['return(%)'] = (SP_all_monthly['S&P_Close']-SP_all_monthly['Close_t-1'])/SP_all_monthly['Close_t-1'] * 100

# Multiclass classification 
SP_all_monthly['return_class'] = SP_all_monthly['return(%)'].apply(lambda x: 1 if x >= 2 
                                              else( 0 if x < 2 and x > -2 else -1))

SP_all_monthly.head(5)

In [ ]:
sns.countplot(x=SP_all_monthly['return_class'], palette = 'Set2')

The goal of S&P index prediction is to predict future('t+1') index based on the current information('t'). Let's create our target variable 'Close_t+1' and 'return rate_t+1'. 

In [ ]:
SP_all_monthly['Close_t+1'] = SP_all_monthly['S&P_Close'].shift(1)
SP_all_monthly['return(%)_t+1'] = SP_all_monthly['return(%)'].shift(1)
SP_all_monthly['return_class_t+1'] = SP_all_monthly['return_class'].shift(1)

In [ ]:
SP_all_monthly['return_class'].value_counts()

In [ ]:
SP_all_monthly.to_csv('SP_all_monthly_binary.csv')

Data explanation: 

`filename` 
* SP_all_monthly : same month return prediction
* SP_all_monthly_t+1 : next month return prediction based on current month information (multiclass)
* SP_all_monthly_binary : next month return prediction based on current month information (binary)
* (SP_all_monthly_t+2 : next month return prediction based on 1-month prior information (multiclass))